In [1]:
import json
import csv
import os
from pprint import pprint as pr
import sqlalchemy as sa
import pandas as pd

In [11]:
BASE_DIR = os.path.abspath(os.path.dirname('../'))

SERVER='192.168.0.245'
SQLSERVER_DATABASE='lookingglass'
SQLSERVER_USER='thecaptain'
SQLSERVER_PASSWORD='99Redbal00ns'
PORT=1433
SQLSERVER_BASE_URI = 'mssql+pyodbc://{USER}:{PWD}@{SERVER}:{PORT}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'

engine = sa.create_engine(SQLSERVER_BASE_URI.format(
  SERVER=SERVER,
  DATABASE=SQLSERVER_DATABASE,
  USER=SQLSERVER_USER,
  PWD=SQLSERVER_PASSWORD,
  PORT=PORT
))

In [12]:
detail_df = pd.read_sql_table(table_name='ReplayDetail', con=engine)
draft_df = pd.read_sql_table(table_name='ReplayDraft', con=engine)
replay_player_df = pd.read_sql_table(table_name='ReplayPlayer', con=engine)
map_df = pd.read_sql_table(table_name='Map', con=engine)
hero_df = pd.read_sql_table(table_name='Hero', con=engine)

In [13]:
print(f'Total ReplayDetail Records - {detail_df.shape[0]}')
print(f'Total ReplayDraft Records - {draft_df.shape[0]}')
print(f'Total ReplayPlayer Records - {replay_player_df.shape[0]}')
print(f'Total Hero Records - {hero_df.shape[0]}')
print(f'Total Map Records - {map_df.shape[0]}')

Total ReplayDetail Records - 15067
Total ReplayDraft Records - 121389
Total ReplayPlayer Records - 150003
Total Hero Records - 90
Total Map Records - 23


In [14]:
map_df = map_df[['MapId', 'MapName']]
hero_df = hero_df[['HeroId', 'HeroName', 'NewRole']]
draft_df = draft_df[['ReplayId', 'HeroId', 'RandomValue', 'PickType', 'DraftIndex', 'Team']]
detail_df = detail_df[['ReplayId', 'SourceReplayId', 'MapId', 'GameType', 'GameDate', 'GameDateFormatted', 'WinningTeam', 'FirstToTen', 'DraftFirstTeam']]

In [15]:
replay_player_detail_df = pd.merge(replay_player_df, detail_df, left_on='ReplayId', right_on='ReplayId')
replay_player_detail_df = pd.merge(replay_player_detail_df, map_df, left_on='MapId', right_on='MapId')
replay_player_detail_df = pd.merge(replay_player_detail_df, map_df, left_on='HeroId', right_on='HeroId')

In [16]:
replay_player_detail_df.head()

,PlayerReplayId,ReplayId,RandomValue,PlayerId,HeroId,HeroLevel,Team,IsWinner,Party,FirstToTen_x,...,SourceReplayId,MapId,GameType,GameDate,GameDateFormatted,WinningTeam,FirstToTen_y,DraftFirstTeam,MapName_x,MapName_y
0,1,1,1531044922,12617,12,15,0,0,0,1,...,0,11,QuickMatch,2021-09-03 12:08:06,2021-09-03,1,0,0,Tomb of the Spider Queen,Tomb of the Spider Queen
1,2,1,1531044922,12618,13,12,0,0,0,1,...,0,11,QuickMatch,2021-09-03 12:08:06,2021-09-03,1,0,0,Tomb of the Spider Queen,Tomb of the Spider Queen
2,3,1,1531044922,12619,74,20,0,0,0,1,...,0,11,QuickMatch,2021-09-03 12:08:06,2021-09-03,1,0,0,Tomb of the Spider Queen,Tomb of the Spider Queen
3,4,1,1531044922,12620,10,14,0,0,0,1,...,0,11,QuickMatch,2021-09-03 12:08:06,2021-09-03,1,0,0,Tomb of the Spider Queen,Tomb of the Spider Queen
4,5,1,1531044922,12621,59,20,0,0,0,1,...,0,11,QuickMatch,2021-09-03 12:08:06,2021-09-03,1,0,0,Tomb of the Spider Queen,Tomb of the Spider Queen


In [63]:
replay_detail_df = pd.merge(detail_df, map_df, left_on='MapId', right_on='MapId')
replay_player_detail_df = pd.merge(replay_player_df, hero_df, left_on='HeroId', right_on='HeroId')
replay_player_detail_df = replay_player_detail_df[]

In [64]:
replay_draft_detail_df = pd.merge(hero_draft_df, map_detail_df, left_on='ReplayId', right_on='ReplayId')

In [65]:
map_detail_df.head()

,ReplayId,SourceReplayId,MapId,GameType,GameDate,GameDateFormatted,WinningTeam,FirstToTen,DraftFirstTeam,MapName
0,1,0,11,QuickMatch,2021-09-03 12:08:06,2021-09-03,1,0,0,Tomb of the Spider Queen
1,13,0,11,Custom,2021-09-07 02:05:11,2021-09-07,1,1,1,Tomb of the Spider Queen
2,16,0,11,QuickMatch,2021-09-08 12:09:48,2021-09-08,0,0,0,Tomb of the Spider Queen
3,19,0,11,StormLeague,2021-09-08 01:39:58,2021-09-08,0,0,0,Tomb of the Spider Queen
4,29,0,11,Custom,2021-09-09 02:38:25,2021-09-09,0,1,0,Tomb of the Spider Queen


In [66]:
hero_draft_df.head()

,ReplayId,HeroId,RandomValue,PickType,DraftIndex,Team,HeroName,NewRole
0,5,58,313964503,Banned,0,-1,Nazeebo,Ranged Assassin
1,13,58,2914723711,Banned,0,-1,Nazeebo,Ranged Assassin
2,14,58,515395070,Banned,0,-1,Nazeebo,Ranged Assassin
3,15,58,3060720725,Banned,1,-1,Nazeebo,Ranged Assassin
4,17,58,456305023,Banned,1,-1,Nazeebo,Ranged Assassin


In [67]:
replay_draft_detail_df.head()

,ReplayId,HeroId,RandomValue,PickType,DraftIndex,Team,HeroName,NewRole,SourceReplayId,MapId,GameType,GameDate,GameDateFormatted,WinningTeam,FirstToTen,DraftFirstTeam,MapName
0,5,58,313964503,Banned,0,-1,Nazeebo,Ranged Assassin,0,6,StormLeague,2021-09-06 08:40:24,2021-09-06,1,1,0,Garden of Terror
1,5,10,313964503,Banned,1,-1,Azmodan,Ranged Assassin,0,6,StormLeague,2021-09-06 08:40:24,2021-09-06,1,1,0,Garden of Terror
2,5,26,313964503,Banned,2,-1,Garrosh,Tank,0,6,StormLeague,2021-09-06 08:40:24,2021-09-06,1,1,0,Garden of Terror
3,5,82,313964503,Banned,3,-1,Valla,Ranged Assassin,0,6,StormLeague,2021-09-06 08:40:24,2021-09-06,1,1,0,Garden of Terror
4,5,83,313964503,Picked,4,0,Varian,Bruiser,0,6,StormLeague,2021-09-06 08:40:24,2021-09-06,1,1,0,Garden of Terror


In [75]:
replay_draft_detail_df.groupby(['HeroName', 'PickType'])['DraftIndex'].count()

HeroName     PickType
Abathur      Banned       403
             Picked       474
Alarak       Banned        43
             Picked       386
Alexstrasza  Banned        36
                         ... 
Zarya        Picked       213
Zeratul      Banned       128
             Picked       406
Zul'jin      Banned       382
             Picked      1069
Name: DraftIndex, Length: 222, dtype: int64